In [1]:
import tensorflow.compat.v1 as tf
tf.compat.v1.disable_eager_execution()

In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display
from seaborn import color_palette
import cv2

In [3]:
_BATCH_NORM_DECAY = 0.9
_BATCH_NORM_EPSILON = 1e-05
_LEAKY_RELU = 0.1

In [4]:
_ANCHORS = [(10, 13), (16, 30), (33, 23),
           (30, 61), (62, 45), (59, 119),
           (116, 90), (156, 198), (373, 326)]

_MODEL_SIZE = (416, 416)

In [5]:
def batch_norm(inputs, training, data_format):
    return tf.layers.batch_normalization(inputs=inputs,
                                        axis = 1 if data_format == 'channels_first' else 3,
                                        momentum= _BATCH_NORM_DECAY, epsilon = _BATCH_NORM_EPSILON, 
                                        scale= True, training=training)

In [6]:
def fixed_padding(inputs, kernel_size, data_format):
    pad_total = kernel_size - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg
    
    if data_format == 'channels_first':
        padded_inputs = tf.pad(inputs, [[0, 0],
                                        [0, 0],
                                        [pad_beg, pad_end],
                                        [pad_beg, pad_end]])
    else:
        padded_inputs = tf.pad(inputs, [[0, 0], [pad_beg, pad_end],
                                        [pad_beg, pad_end], [0, 0]])
        
    return padded_inputs                

In [7]:
def conv2_fixed_padding(inputs, filters, kernel_size, data_format, strides = 1):
    if strides > 1:
        inputs = fixed_padding(inputs, kernel_size, data_format)
        
    return tf.layers.conv2d(inputs=inputs, filters=filters, kernel_size= kernel_size,
                           strides= strides, padding=("SAME" if strides == 1 else "VALID"),
                           use_bias=False, data_format = data_format)

In [8]:
def darknet_53_block(inputs, filters, trainig, data_format, strides = 1):
    shortcut = inputs
    
    inputs = conv2_fixed_padding(inputs=inputs, filters=filters, kernel_size=1, strides=strides, data_format=data_format)
    inputs = batch_norm(inputs=inputs, trainig =trainig, data_format = data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    
    
    inputs = conv2_fixed_padding(inputs=inputs, filters=2*filters, kernel_size= 3, strides=strides, data_format = data_format)
    inputs = batch_norm(inputs=inputs, trainig =trainig, data_format = data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    
    inputs+= shortcut
    
    return inputs

In [9]:
def darknet53(inputs, training, data_format):
    
    inputs = conv2_fixed_padding(inputs=inputs, filters=32, kernel_size=3, data_format=data_format)
    inputs = batch_norm(inputs=inputs, trainig =trainig, data_format = data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    
    
    inputs = conv2_fixed_padding(inputs=inputs, filters=64, kernel_size=3, data_format=data_format, strides=2)
    inputs = batch_norm(inputs=inputs, trainig =trainig, data_format = data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    
    inputs = darknet_53_block(inputs=inputs, filters=32,trainig=trainig, data_format=data_format)
    
    
    inputs = conv2_fixed_padding(inputs=inputs, filters=128, kernel_size=3, data_format=data_format, strides=2)
    inputs = batch_norm(inputs=inputs, trainig =trainig, data_format = data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    
    
    for _ in range(8):
        inputs = darknet_53_block(inputs=inputs, filters=128,trainig=trainig, data_format=data_format)
    
    route1 = inputs
    
    inputs = conv2_fixed_padding(inputs=inputs, filters=512, kernel_size=3, data_format=data_format, strides=2)
    inputs = batch_norm(inputs=inputs, trainig =trainig, data_format = data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    
    
    for _ in range(8):
        inputs = darknet_53_block(inputs=inputs, filters=256,trainig=trainig, data_format=data_format)
    
    route2 = inputs
    
    inputs = conv2_fixed_padding(inputs=inputs, filters=1024, kernel_size=3, data_format=data_format, strides=2)
    inputs = batch_norm(inputs=inputs, trainig =trainig, data_format = data_format)
    inputs = tf.nn.leaky_relu(inputs, alpha=_LEAKY_RELU)
    
    
    
    for _ in range(4):
        inputs = darknet_53_block(inputs=inputs, filters=512,trainig=trainig, data_format=data_format)
    
    return route1, route2, inputs
    